In [6]:
import os
import requests
import logging
from datetime import datetime

In [7]:
import pandas as pd
import requests

# Fetch the data.
df = pd.read_csv("https://ourworldindata.org/grapher/total-alcohol-consumption-per-capita-litres-of-pure-alcohol.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
df.head()

,entity,code,year,sh_alc_pcap_li
0,Afghanistan,AFG,2000,0.003
1,Afghanistan,AFG,2001,0.003
2,Afghanistan,AFG,2002,0.007
3,Afghanistan,AFG,2003,0.016
4,Afghanistan,AFG,2004,0.021


In [20]:
years_lsit = list(df['year'].unique())
df_pvt = df.pivot(index='entity', columns='year', values='sh_alc_pcap_li')
df_pvt['index'] = df_pvt[years_lsit].sum(axis=1)
df_pvt = df_pvt.sort_values(by='index', ascending=False)
df_pvt.head()

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,index
entity,,,,,,,,,,,,,,,,,,,,,
Romania,19.05,19.05,19.40,19.29,18.69,18.32,18.09,18.34,18.39,18.13,...,17.18,17.09,16.98,16.85,16.82,16.92,16.99,16.99,16.80,374.31
Uganda,17.79,17.79,17.58,17.34,17.22,17.16,17.08,16.07,15.01,14.11,...,13.65,13.47,13.38,13.28,12.89,12.58,12.21,12.21,11.30,310.04
Czechia,14.22,14.22,14.21,14.13,14.09,13.99,14.11,14.10,14.04,13.67,...,13.08,13.14,13.11,13.22,13.20,13.26,13.29,13.29,11.99,284.82
Estonia,10.45,10.45,11.12,12.50,13.64,15.25,15.75,15.84,15.09,14.42,...,15.45,15.71,15.64,15.06,13.92,12.33,11.26,11.26,10.68,284.78
Georgia,10.57,10.57,10.21,10.46,11.09,12.26,13.53,14.52,14.70,14.80,...,14.67,14.22,14.02,13.95,13.97,13.97,14.33,14.33,14.41,280.41


In [21]:
generations = {
    'Gen X': range(1965, 1981), # 35 years - 19 years
    'Gen Y': range(1981, 1997), # 19 years - 3 years
    'Gen Z': range(1997, 2010), # 3 years
    'Gen Alpha': range(2010, 2025),
    'Gen Beta': range(2025, 2040)
}

In [22]:
import plotly.express as px
import plotly.graph_objects as go

# 1. Select Major 5 Countries (Top 5 by total consumption over the period)
# Ensure we use the pivoted df to get the index of top 5
top_5_countries = df_pvt.head(5).index.tolist()

# 2. Filter Data for 2000-2020 and Top 5 Countries
# We need the original 'df' for plotting with px
df_filtered = df[
    (df['entity'].isin(top_5_countries)) & 
    (df['year'] >= 2000) & 
    (df['year'] <= 2020)
].sort_values('year')

# 3. Create Line Chart
fig = px.line(
    df_filtered, 
    x='year', 
    y='sh_alc_pcap_li', 
    color='entity',
    title='Alcohol Consumption per Capita (2000-2020) - Top 5 Countries',
    labels={'sh_alc_pcap_li': 'Litres of Pure Alcohol', 'year': 'Year', 'entity': 'Country'},
    markers=True
)

# 4. Add Annotations for Generations (Birth Years)
# We only annotate generations that have birth years overlapping with the chart range (2000-2020)
# Gen Z: 1997 - 2010 (Overlaps 2000-2010)
# Gen Alpha: 2010 - 2025 (Overlaps 2010-2020)

# Define overlap colors
gen_colors = {
    'Gen Z': 'rgba(255, 0, 0, 0.1)',      # Reddish
    'Gen Alpha': 'rgba(0, 0, 255, 0.1)',  # Bluish
}

for gen_name, year_range in generations.items():
    start_year = year_range.start
    end_year = year_range.stop
    
    # Check if generation interval overlaps with chart range [2000, 2020]
    # Intersection logic
    chart_start, chart_end = 2000, 2020
    overlap_start = max(start_year, chart_start)
    overlap_end = min(end_year, chart_end)
    
    if overlap_start < overlap_end:
        # Add shaded region
        fig.add_vrect(
            x0=overlap_start, 
            x1=overlap_end,
            fillcolor=gen_colors.get(gen_name, 'rgba(100, 100, 100, 0.1)'),
            opacity=0.5,
            layer="below", 
            line_width=0,
            annotation_text=f"{gen_name} Born",
            annotation_position="top left"
        )

fig.show()